# Toward an ARABIC Stop-Words List Generation
One of the most important preprocessing techniques is the removal of functional words which affects the performance of text mining tasks. In this [paper](https://www.researchgate.net/publication/306364790_Toward_an_ARABIC_Stop-Words_List_Generation), a statistical approach is presented to extract Arabic stop-words list. The extracted list was compared to a general list. The comparison yield an improvement in an ANN based classifier using the generated stop-words list over the general list.


stop words are words which are filtered out before or after processing of natural language data.

In [109]:
import pandas as pd
import glob

import unicodedata as ud

import numpy as np
import pickle
import matplotlib.pyplot as plt
from collections import Counter

from sklearn import feature_extraction, model_selection

### Explore Data

In [73]:
#get all csv files and concatenate them into single dataframe
path = r'../data/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
data.head(n=-1)

,article
0,بمجرد أن تكون لك عملية مالية أو غيرها داخل أغل...
1,قبل فترة ليست بالقصيرة صدرت قرارات بخصوص فتح ا...
2,هناك مثل شعبي عقلاني يقول (راحت السكرة وجت الف...
3,انتهت قبل عدة أيام مناسك الحج التي أداها الحجا...
4,إن لانتشار الأشجار في المدن مزايا عديدة، منها ...
...,...
23792,تعيد شبكات التواصل فكرة الساحة العامة الرئيسية...
23793,كتب داليبور روهاك، باحث زميل في معهد اميركان ا...
23794,مضى الآن على مقاطعة الدول الأربع لقطر ما يزيد ...
23795,ما بين البَصْرة ونجد، في عشرينيات القرن الماضي...


In [107]:
#remove NaN
data = data[data['article'].notna()]

In [114]:
#remove punctuation
for i in range(data.shape[0]):
    data.iloc[i][0] = ''.join(c for c in data.iloc[i][0] if not ud.category(c).startswith('P'))
data

,article
0,بمجرد أن تكون لك عملية مالية أو غيرها داخل أغل...
1,قبل فترة ليست بالقصيرة صدرت قرارات بخصوص فتح ا...
2,هناك مثل شعبي عقلاني يقول راحت السكرة وجت الفك...
3,انتهت قبل عدة أيام مناسك الحج التي أداها الحجا...
4,إن لانتشار الأشجار في المدن مزايا عديدة منها ز...
...,...
23793,كتب داليبور روهاك باحث زميل في معهد اميركان ان...
23794,مضى الآن على مقاطعة الدول الأربع لقطر ما يزيد ...
23795,ما بين البَصْرة ونجد في عشرينيات القرن الماضي ...
23796,في محاضرته واجب الشباب المسلم اليوم التي ألقاه...


In [115]:
words = pd.DataFrame()
for i in range(data.shape[0]):
    words = words.append(data.iloc[i][0].split())
words

,0
0,بمجرد
1,أن
2,تكون
3,لك
4,عملية
...,...
339,الفساد
340,جعجعة
341,ولا
342,يرى


In [124]:
wordsFrequency = pd.DataFrame(Counter(words[0]).most_common(50), columns=['word', 'frequency'])
wordsFrequency

,word,frequency
0,في,356815
1,من,286550
2,على,172141
3,أن,146032
4,إلى,99883
5,التي,80657
6,عن,67910
7,لا,65857
8,ما,65689
9,أو,54974
